# Tratamento da base de dados

In [2]:
import pandas as pd
import numpy as np

base_cases = pd.read_csv('house_prices.csv')

In [3]:
base_cases

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [4]:
base_cases.drop(columns = ['id', 'date', 'sqft_living15', 'sqft_lot15'], axis=1, inplace=True)

In [7]:
base_cases.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long'],
      dtype='object')

In [9]:
base_treinamento = base_cases.iloc[0:15129, :]
base_treinamento.shape

(15129, 17)

In [10]:
base_teste = base_cases.iloc[15129:, :]
base_teste.shape

(6484, 17)

In [13]:
base_teste.shape[0] + base_treinamento.shape[0] 

21613

In [14]:
X_teste = base_teste.iloc[:,1:17].values
# formado array
X_teste

array([[ 4.00000e+00,  2.75000e+00,  2.58000e+03, ...,  9.81150e+04,
         4.76753e+01, -1.22304e+02],
       [ 4.00000e+00,  2.50000e+00,  3.20000e+03, ...,  9.80590e+04,
         4.75273e+01, -1.22143e+02],
       [ 3.00000e+00,  1.00000e+00,  8.40000e+02, ...,  9.81780e+04,
         4.74940e+01, -1.22275e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [15]:
y_teste = base_teste.iloc[:,0].values
y_teste

array([937750., 725126., 135000., ..., 402101., 400000., 325000.])

In [17]:
# tranformando em csv os dados e retirando 
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)

In [18]:
# tranformando em csv os dados e retirando 
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)

# Configurações Sage

In [19]:
import sagemaker
import boto3
from sagemaker import Session

In [23]:
session = sagemaker.Session()
bucket = 'cursoawssagemakersp'
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_data_set = 'datasets/house-prices'
# nome da base de dados de treinamento
key_train = 'houses-train-data-xgboost'
key_test = 'houses-test-data-xgboost'
# permissões
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_data_set, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_data_set, key_test)
# onde o modelo será armazenado
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

Role: arn:aws:iam::605390214759:role/service-role/AmazonSageMaker-ExecutionRole-20230311T091643
Localização da base de treinamento: s3://cursoawssagemakersp/datasets/house-prices/train/houses-train-data-xgboost
Localização da base de teste: s3://cursoawssagemakersp/datasets/house-prices/test/houses-test-data-xgboost
Modelo final será salvo em: s3://cursoawssagemakersp/modelos/house-prices/xgboost/output


In [ ]:
session = sagemaker.Session()
bucket = 'cursoawssagemaker'
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_dataset = 'datasets/house-prices'
key_train = 'houses-train-data-xgboost'
key_test = 'houses-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

In [28]:
import os

# salvar os arquivos no bucket em formula de binário no train
with open('house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_data_set, 'train', key_train))
    .upload_fileobj(f)
    

In [27]:
import os

# salvar os arquivos no bucket 
with open('house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_data_set, 'test', key_test)).upload_fileobj(f)
    

# Treinamento do XGBoost

In [39]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [41]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgboost = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)

In [42]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgboost.set_hyperparameters(num_round = 100)

In [43]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
# para unir essas duas informações
# a cada etapa do algortimo será feita a validação
data_channels = {'train': train_input, 'validation': validation_input}

In [44]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2023-03-14-21-52-30-161


2023-03-14 21:52:30 Starting - Starting the training job...
2023-03-14 21:52:51 Starting - Preparing the instances for training......
2023-03-14 21:53:50 Downloading - Downloading input data...
2023-03-14 21:54:30 Training - Training image download completed. Training in progress....
2023-03-14 21:54:56 Uploading - Uploading generated training modelArguments: train
[2023-03-14:21:54:48:INFO] Running standalone xgboost training.
[2023-03-14:21:54:48:INFO] File size need to be processed in the node: 1.55mb. Available memory size in the node: 23692.42mb
[2023-03-14:21:54:48:INFO] Determined delimiter of CSV input is ','
[21:54:48] S3DistributionType set as FullyReplicated
[21:54:48] 15129x16 matrix with 242064 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-03-14:21:54:48:INFO] Determined delimiter of CSV input is ','
[21:54:48] S3DistributionType set as FullyReplicated
[21:54:48] 6484x16 matrix with 103744 entries loaded from /opt/ml/input/data/va

# Deplot, previsões e avaliação

In [46]:
xgboost_regressor = xgboost.deploy(initial_instance_count=1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-03-14-22-05-50-735
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-03-14-22-05-50-735
INFO:sagemaker:Creating endpoint with name xgboost-2023-03-14-22-05-50-735


------!

In [47]:
from sagemaker.serializers import CSVSerializer

# configurando o tipo de entrada
xgboost_regressor.serializer = CSVSerializer()

In [48]:
X_teste.shape, type(X_teste)

((6484, 16), numpy.ndarray)

In [57]:
previsoes = np.array(xgboost_regressor.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

array([715760.25 , 759194.25 , 125879.664, ..., 298663.34 , 468468.56 ,
       292532.9  ], dtype=float32)

In [59]:
previsoes.shape, y_teste.shape

((6484,), (6484,))

In [60]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
print('MAE = ', mae, '\nMSE = ', mse)

MAE =  69311.54741358536 
MSE =  14874845482.575417


# Tuning

In [61]:
#eta = numero de passos para previnir o overfitting
# alpha = regularização, é baseada nos pesos extras para evitar o overfitting
#min_child_weight = numero de min de instancia para a folha da arvore
#max_depth = profundidade de arvore
# ResourceLimits para poder computacional 
# MaxNumberOfTrainingJobs é o numero de algortimo que será gerado
# MaxParallelTrainingJobs numero de algoritmo que será executado em paralelo

In [62]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "50",
          "Name": "num_round"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 9,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:rmse",
      "Type": "Minimize"
    }
  }

In [63]:
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_train_data
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_test_data
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,subpasta_modelo)
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.c4.2xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "eval_metric": "rmse",
      "objective": "reg:linear",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}

In [65]:
smclient = boto3.client('sagemaker')

In [68]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = "xgboosttuninghouses2",
                                          HyperParameterTuningJobConfig = tuning_job_config,
                                          TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:sa-east-1:605390214759:hyper-parameter-tuning-job/xgboosttuninghouses2',
 'ResponseMetadata': {'RequestId': '009032b0-9b5d-43d6-a7a4-04e135f41dc5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '009032b0-9b5d-43d6-a7a4-04e135f41dc5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Tue, 14 Mar 2023 22:46:16 GMT'},
  'RetryAttempts': 0}}

In [69]:
xgboost.set_hyperparameters(num_round = 219, 
                            alpha = 1.59, 
                            eta = 0.07, 
                            min_child_weight = 2.406,
                            rate_drop = 0.3,
                            tweedie_varience_power = 1.4)

In [70]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
# para unir essas duas informações
# a cada etapa do algortimo será feita a validação
data_channels = {'train': train_input, 'validation': validation_input}

In [71]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2023-03-14-22-53-18-575


2023-03-14 22:53:18 Starting - Starting the training job...
2023-03-14 22:53:35 Starting - Preparing the instances for training......
2023-03-14 22:54:31 Downloading - Downloading input data...
2023-03-14 22:55:11 Training - Training image download completed. Training in progress...Arguments: train
[2023-03-14:22:55:29:INFO] Running standalone xgboost training.
[2023-03-14:22:55:29:INFO] File size need to be processed in the node: 1.55mb. Available memory size in the node: 23679.0mb
[2023-03-14:22:55:29:INFO] Determined delimiter of CSV input is ','
[22:55:29] S3DistributionType set as FullyReplicated
[22:55:29] 15129x16 matrix with 242064 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-03-14:22:55:29:INFO] Determined delimiter of CSV input is ','
[22:55:29] S3DistributionType set as FullyReplicated
[22:55:29] 6484x16 matrix with 103744 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[22:55:29] src/tree/up

In [72]:
xgboost_regressor = xgboost.deploy(initial_instance_count=1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-03-14-22-56-07-659
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-03-14-22-56-07-659
INFO:sagemaker:Creating endpoint with name xgboost-2023-03-14-22-56-07-659


------!

In [73]:
# configurando o tipo de entrada
xgboost_regressor.serializer = CSVSerializer()

In [74]:
previsoes = np.array(xgboost_regressor.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

array([730289.06, 803608.4 , 145735.12, ..., 322631.8 , 442199.56,
       322340.22], dtype=float32)

In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
print('MAE = ', mae, '\nMSE = ', mse)

MAE =  67160.80174516117 
MSE =  14628484352.380383
